# A First Look at the SDSS Photometric "Galaxy" Catalog

* The Sloan Digital Sky Survey imaged over 10,000 sq degrees of sky (about 25% of the total), automatically detecting, measuring and cataloging millions of "objects".


* While the primary data products of the SDSS was(and still are) its spectroscopic surveys, the photometric survey provides an important testing ground for dealing with pure imaging surveys like those being carried out by DES and that is planned with LSST.


* Let's download part of the SDSS photometric object catalog and explore it.

SDSS data release 12 (DR12) is described [at the SDSS3 website](http://www.sdss.org/dr12/) and in the survey paper by [Alam et al 2015](http://arxiv.org/abs/1501.00963). 


We will use the [SDSS DR12 SQL query interface](http://skyserver.sdss.org/dr12/en/tools/search/sql.aspx). For help designing queries, the [sample queries page](http://skyserver.sdss.org/dr12/en/help/docs/realquery.aspx) is invaluable, and you will probably want to check out the links to the ["schema browser"]() at some point as well. Notice the "check syntax only" button on the SQL query interface: this is very useful for debugging SQL queries.


> _Small test queries can be executed directly in the browser. Larger ones (involving more than a few tens of thousands of objects, or that involve a lot of processing) should be submitted via the [CasJobs](http://skyserver.sdss.org/CasJobs/) system. Try the browser first, and move to CasJobs when you need to._

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import numpy as np
import SDSS
import pandas as pd
import matplotlib
%matplotlib inline

In [16]:
objects = "SELECT top 1000 \
ra, \
dec, \
dered_u as u, \
dered_g as g, \
dered_r as r, \
dered_i as i, \
petroR50_i AS size \
FROM PhotoObjAll \
WHERE \
(type = '3' OR type = '6' AND \
 ra > 185.00 AND ra < 185.05 AND \
 dec > 15.00 and dec < 15.05)"
print objects

SELECT top 1000 ra, dec, dered_u as u, dered_g as g, dered_r as r, dered_i as i, petroR50_i AS size FROM PhotoObjAll WHERE (type = '3' OR type = '6' AND  ra > 185.00 AND ra < 185.05 AND  dec > 15.00 and dec < 15.05)


In [17]:
# Download data. This can take a while...
data = SDSS.select(objects)
data

,ra,dec,u,g,r,i,size
0,336.438801,-0.834257,10.72432,9.421398,12.27091,9.184780,3.183596
1,336.438777,-0.834246,10.70998,9.475405,12.86676,9.218829,3.065464
2,336.439673,-0.837215,20.09617,14.048700,13.99016,15.419450,20.184450
3,336.438802,-0.834269,10.73401,9.411324,12.02491,9.174438,3.140940
4,336.438796,-0.834273,10.72197,9.422896,12.14973,9.195747,2.944434
5,336.439637,-0.837205,23.61069,14.103120,13.84958,13.373510,8.316065
6,336.442522,-0.830764,20.64058,19.737190,19.32571,22.258610,0.686464
7,336.444242,-0.832926,21.34084,22.898090,21.08305,23.250070,0.896665
8,336.451245,-0.822988,23.93277,20.587270,19.84060,19.789320,-9999.000000
9,336.448259,-0.843711,22.87791,21.713160,20.78844,21.395220,0.744543


In [19]:
!mkdir -p downloads
data.to_csv("downloads/SDSSobjects.csv")

## Visualizing Data in N-dimensions

This is, in general, difficult.